In [17]:
import pandas as pd
import sqlite3
import openai
# import json
import os
# from datetime import datetime
from dotenv import load_dotenv

# Load the environment variables
load_dotenv()

True

In [19]:
# Load your OpenAI API key
openai.api_key = os.getenv("OPENAI_API_KEY")
# print(openai.api_key)

In [14]:
# Create SQLite in-memory DB or file-based
conn = sqlite3.connect('orders.db')  # or use ':memory:' for temporary in-memory DB

In [6]:
pd.read_sql_query("SELECT DISTINCT `Transaction Type` FROM orders", conn)

,Transaction Type
0,Shipment
1,Cancel
2,Refund
3,FreeReplacement


In [7]:
DATABASE_SCHEMA = """Here are the columns of the database table 'orders':

['Seller Gstin', 'Invoice Number', 'Invoice Date', 'Transaction Type',
       'Order Id', 'Shipment Id', 'Shipment Date', 'Order Date',
       'Shipment Item Id', 'Quantity', 'Item Description', 'Asin', 'Hsn/sac',
       'Sku', 'Product Tax Code', 'Bill From City', 'Bill From State',
       'Bill From Country', 'Bill From Postal Code', 'Ship From City',
       'Ship From State', 'Ship From Country', 'Ship From Postal Code',
       'Ship To City', 'Ship To State', 'Ship To Country',
       'Ship To Postal Code', 'Invoice Amount', 'Tax Exclusive Gross',
       'Total Tax Amount', 'Cgst Rate', 'Sgst Rate', 'Utgst Rate', 'Igst Rate',
       'Compensatory Cess Rate', 'Principal Amount', 'Principal Amount Basis',
       'Cgst Tax', 'Sgst Tax', 'Igst Tax', 'Utgst Tax',
       'Compensatory Cess Tax', 'Shipping Amount', 'Shipping Amount Basis',
       'Shipping Cgst Tax', 'Shipping Sgst Tax', 'Shipping Utgst Tax',
       'Shipping Igst Tax', 'Shipping Cess Tax Amount', 'Gift Wrap Amount',
       'Gift Wrap Amount Basis', 'Gift Wrap Cgst Tax', 'Gift Wrap Sgst Tax',
       'Gift Wrap Utgst Tax', 'Gift Wrap Igst Tax',
       'Gift Wrap Compensatory Cess Tax', 'Item Promo Discount',
       'Item Promo Discount Basis', 'Item Promo Tax',
       'Shipping Promo Discount', 'Shipping Promo Discount Basis',
       'Shipping Promo Tax', 'Gift Wrap Promo Discount',
       'Gift Wrap Promo Discount Basis', 'Gift Wrap Promo Tax',
       'Tcs Cgst Rate', 'Tcs Cgst Amount', 'Tcs Sgst Rate', 'Tcs Sgst Amount',
       'Tcs Utgst Rate', 'Tcs Utgst Amount', 'Tcs Igst Rate',
       'Tcs Igst Amount', 'Warehouse Id', 'Fulfillment Channel',
       'Payment Method Code', 'Credit Note No', 'Credit Note Date']
"""

In [46]:
# Function to generate business logic
def get_business_logic(question):
    system_prompt = f"""
You are a business analyst assistant.

Schema:
{DATABASE_SCHEMA}

Instructions:
1. Do NOT write SQL.
2. Convert the question into step-by-step business logic.
3. Include filters, joins, or groupings as needed.
4. Format your output as a clearly numbered list, one step per line.

In response to the question asked by the user, return the business logic steps in plain English.
"""
    response = openai.chat.completions.create(
    model="gpt-4",
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": question}
    ]

    )
    # print(f"response: {response}")
    logic = response.choices[0].message.content
    # print("Business Logic:\n", logic)
    return logic
    # return response

In [41]:
# Ask a question
r = get_business_logic("What is the total amount of orders in August?")

response: ChatCompletion(id='chatcmpl-BU9xea5jrGcxbqahsQHdKfT7F9aYd', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="1. Select the 'orders' table.\n2. Filter the records where 'Order Date' falls within the month of August. This can be done by checking whether the month part of the 'Order Date' is August.\n3. After this filter, select the 'Invoice Amount' column.\n4. Sum up over all the values of 'Invoice Amount'. This will give us the total amount of orders in August.", refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None, annotations=[]))], created=1746527830, model='gpt-4-0613', object='chat.completion', service_tier='default', system_fingerprint=None, usage=CompletionUsage(completion_tokens=85, prompt_tokens=567, total_tokens=652, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=Prompt

In [43]:
# r.__dict__

In [44]:
# Load questions
df = pd.read_csv("questions.csv")  # expects a column called 'question'

In [47]:
# Use only the first 2 rows
# test_df = df.head(2)

# Iterate and print nicely
for i, row in df.iterrows():
    question = row["question"]
    logic = get_business_logic(question)

    print(f"\n\033[1mQuestion {i+1}:\033[0m {question}\n")
    for line in logic.split("\n"):
        print(line.strip())
    print("\n" + "-" * 80)


Question 1: How many orders did I have in November 2021

1. Start with the 'orders' table.
2. Apply filter on the 'Order Date' column, to include only the dates that fall in the month of November 2021.
3. Count the number of unique 'Order Id' values in the filtered table. Each unique 'Order Id' corresponds to an individual order.
4. The final count will represent the total number of orders in November 2021.

--------------------------------------------------------------------------------

Question 2: How many orders got refunded

1. Start by identifying the relevant column(s) or variable(s) in the dataset which marks an order as refunded.
2. It is likely that refunded orders would be reflected under the 'Transaction Type' column if such order statuses are recorded in the dataset. Alternatively, there could be a 'Payment Method Code' or 'Credit Note No' relevant for this purpose.
3. Filter the dataset where 'Transaction Type' equals 'Refund' or where 'Credit Note No' is not null, or wh